## Lecture 13: Regression Models

We want to develop techniques which allow us to objectively select a good model for fitting the data.

In [ ]:
import numpy as np
import os

import matplotlib.pyplot as plt
from matplotlib import rc

plt.rcParams['xtick.labelsize']=16      # change the tick label size for x axis
plt.rcParams['ytick.labelsize']=16      # change the tick label size for x axis
plt.rcParams['axes.linewidth']=1        # change the line width of the axis
plt.rcParams['xtick.major.width'] = 3   # change the tick line width of x axis
plt.rcParams['ytick.major.width'] = 3   # change the tick line width of y axis
rc('text', usetex=False)                # disable LaTeX rendering in plots
rc('font',**{'family':'DejaVu Sans'})   # set the font of the plot to be DejaVu Sans

Deciding what set of parameters to optimize over will play a foundational role in extracting interpretable results and meaningful models from data.

In the previous few lectures, we explored the interplay of optimizing over $l_2$ and $l_1$ norms.

### 1. Generate some synthetic data for a parabola

Now, to illustrate further the variety of possible outcomes, we consider the following simple example of synthetic data:

$$ f(x) = x^2 + \mathcal{N}(0, \sigma) $$

Here, $f(x)$ represents noisy measurements of a parabola, and $\mathcal{N}(0, \sigma)$ is a normally distributed random variable with mean zero and standard deviation $\sigma$.

In [ ]:
x = np.linspace(0, 4, 100)
mu = 0.0
sigma = 0.1
M = 20
f = np.zeros((100, M))
for i in range(M):
  f[:,i] = x**2 + np.random.normal(mu, sigma, *x.shape)

fig = plt.figure(figsize=(10, 4), dpi = 80)
plt.plot(x, f)
plt.show()

Here we show a plot of 20 different measurements with 100 datapoints each for $f(x)$ with a standard deviation of $0.1$. Based on the plot, you can see there is very little deviation.

Now, let's try to fit a single measurements to a curve with the shape of $x^p$.  The fit would be trivial if we know that $p=2$, but here, we do not know this information *a priori*.

### 2. Perform Regression for different power of $x$

Let's try to do regression on the data on four different measurements independently, fitting to $x^p$ where $p\in[0, 19]$.

(See white board for regression construct).

In [ ]:
### build matrix A to perform regression
A = np.zeros((len(x), M))
for j in range(M):
  A[:,j] = x**j

fig, axs = plt.subplots(2,2)
axs = axs.reshape(-1)
fig.set_size_inches(10, 7)

for j in range(4):
  f = x**2 + np.random.normal(mu, sigma, *x.shape)
  A_inv = np.linalg.pinv(A) # Least-square fit
  x_apprx = A_inv @ f
  f_apprx = A @ x_apprx
  E2 = np.linalg.norm(f_apprx-f, ord = 2) / np.linalg.norm(f,ord=2)
  axs[j].bar(range(len(x_apprx)), x_apprx)
  axs[j].set_title("L2 Norm Relative Error is: "+str("%.4f" % E2))

plt.show()

Let's see how our model compares to the original data.

In [ ]:
fig, axs = plt.subplots(2,2)
axs = axs.reshape(-1)
fig.set_size_inches(10, 7)

for j in range(4):
  f = x**2 + np.random.normal(mu, sigma, *x.shape)
  A_inv = np.linalg.pinv(A) # Least-square fit
  x_apprx = A_inv @ f
  f_apprx = A @ x_apprx
  axs[j].plot(x, f, label = "raw data")
  axs[j].plot(x, f_apprx, label = "fit data")
  axs[j].legend()

plt.show()

### 3. Now let's try different kinds of regression procedures

Different regression procedures:
1. **least-squares regression** (`pinv`) (what we have been doing so far)

  Moore-Penrose pseudo-inverse (`pinv`) set $\lambda_1 = \lambda_2 = 0$
2. **the backslash operator** (` \ `)

  The backslash command (` \ `) for over-determined systems solves the linear system via a $QR$ decomposition
3. **least absolute shrinkage and selection operator** LASSO (`lasso`)

  The LASSO (`lasso`) set $\lambda_1 > 0$ and $\lambda_2 = 0$.
4. **robust fit** (`robustfit`)

  Robust fit does weighted least-squares fitting. It allows one to leverage robust statistics methods and penalize according to the Huber norm so as to promote outlier rejection
5. **ridge regression** (`ridge`)

  Ridge regression (ridge) solves (4.40) with $\lambda_1 = 0$ and $\lambda_2 > 0$.

Linear Models documentation for `sckit-learn`: https://scikit-learn.org/stable/modules/linear_model.html

In [ ]:
from sklearn import linear_model
import warnings
warnings.filterwarnings('ignore')

## Different regressions
lam = 0.1
E1 = np.zeros(100)
E2 = np.zeros(100)
E3 = np.zeros(100)
E4 = np.zeros(100)
E5 = np.zeros(100)
E6 = np.zeros(100)

X1 = np.zeros((M,100))
X2 = np.zeros((M,100))
X3 = np.zeros((M,100))
X4 = np.zeros((M,100))
X5 = np.zeros((M,100))
X6 = np.zeros((M,100))

for j in range(100):
  f = x**2 + np.random.normal(mu, sigma, *x.shape)

  ### least-square regression
  a1 = np.linalg.pinv(A)
  x1 = a1 @ f
  f1 = A @ x1
  E1[j] = np.linalg.norm(f-f1, ord=2)/np.linalg.norm(f, ord=2)

  ### backslash command (QR decomposition)
  x2 = np.linalg.lstsq(A, f, rcond = None)[0]
  f2 = A @ x2
  E2[j] = np.linalg.norm(f-f2, ord=2)/np.linalg.norm(f, ord=2)

  ### LASSO
  reg3 = linear_model.Lasso(alpha = lam)
  reg3.fit(A, f)
  x3 = reg3.coef_
  f3 = A @ x3
  E3[j] = np.linalg.norm(f-f3, ord=2)/np.linalg.norm(f, ord=2)

  ### Combining l1 and l2 penalization
  reg4 = linear_model.ElasticNet(alpha = 0.8, l1_ratio=lam)
  reg4.fit(A, f)
  x4 = reg4.coef_
  f4 = A @ x4
  E4[j] = np.linalg.norm(f-f4, ord=2)/np.linalg.norm(f, ord=2)

  ### huber/robustfit
  # matlab's robustfit() does not have an exact sklearn analogue
  huber = linear_model.HuberRegressor()
  huber.fit(A, f)
  x5 = huber.coef_
  f5 = A @ x5
  E5[j] = np.linalg.norm(f-f5, ord=2)/np.linalg.norm(f, ord=2)

  ridge = linear_model.Ridge(alpha=1.0)
  ridge.fit(A, f)
  x6 = ridge.coef_
  f6 = A @ x6
  E6[j] = np.linalg.norm(f-f6, ord=2)/np.linalg.norm(f, ord=2)


  X1[:,j] = x1
  X2[:,j] = x2
  X3[:,j] = x3
  X4[:,j] = x4
  X5[:,j] = x5
  X6[:,j] = x6

Err = np.column_stack((E1,E2,E3,E4,E5,E6))

Boxplot explanation: https://vita.had.co.nz/papers/boxplots.pdf

In [ ]:
plt.rcParams['figure.figsize'] = [12, 18]
fig,axs = plt.subplots(3,2)
axs = axs.reshape(-1)

axs[0].boxplot(X1.T, labels = range(M))
axs[0].set_title('pinv', fontsize = 16)
axs[1].boxplot(X2.T, labels = range(M))
axs[1].set_title('lstsq', fontsize = 16)
axs[2].boxplot(X3.T, labels = range(M))
axs[2].set_title('LASSO', fontsize = 16)
axs[3].boxplot(X4.T, labels = range(M))
axs[3].set_title('ElasticNet', fontsize = 16)
axs[4].boxplot(X5.T, labels = range(M))
axs[4].set_title('huber', fontsize = 16)
axs[5].boxplot(X6.T, labels = range(M))
axs[5].set_title('ridge', fontsize = 16)

plt.rcParams['figure.figsize'] = [8, 8]

plt.figure()
plt.boxplot(Err, labels = ['pinv', 'lstsq', 'LASSO', 'ElasticNet', 'huber', 'ridge'])

plt.show()

### 4. Use Pseudo-Inverse as an example to see how error changes as a function of number of powers of $x$ in the model

In [ ]:
M = 10
En = np.zeros((100,M))
A = np.zeros((len(x),M))
f = x**2
for jj in range(M):
  for j in range(jj):
    A[:,j] = x**j
  for j in range(100):
    fn = x**2 + np.random.normal(mu, sigma, *x.shape)
    A_inv = np.linalg.pinv(A)
    X_apprx = A_inv @ fn
    f_apprx = A @ X_apprx
    En[j,jj] = np.linalg.norm(f-f_apprx, ord=2)/np.linalg.norm(f, ord=2)

plt.boxplot(En[:, :], labels = range(0, M))

plt.show()

### 5. In-class exercise

Compute the error as a function of power of x for least-square solver and LASSO